In [2]:
import os
from dotenv import load_dotenv


In [7]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY", "your_openai_api_key_here")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY", "your_langchain_api_key_here")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT", "your_langchain_project_here")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY", "your_groq_api_key_here")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.getenv("GROQ_API_KEY", "your_groq_api_key_here")


'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'

In [10]:
from langchain_groq import ChatGroq

model=ChatGroq(model="qwen-qwq-32b")
result=model.invoke("tell me a joke about cats")
print(result)

content='\n<think>\nOkay, the user wants a joke about cats. Let me think of some common cat-related humor. Cats are known for their playful antics, napping a lot, and sometimes being a bit aloof. Maybe something involving their behavior with lasers, since cats love chasing laser pointers but can\'t catch them. Wait, I should make sure it\'s not too similar to common ones. Hmm, maybe a pun? Or a play on words related to their habits.\n\nAlternatively, maybe a joke that combines a cat\'s特性 with something unexpected. Like, "Why did the cat sit on the computer?" Because it wanted to keep an eye on the mouse! Oh, that\'s a classic one, but maybe they want something newer. Let me check if that\'s overused. Alternatively, how about involving a cat and a computer mouse? Yeah, that\'s a classic pun. Maybe that\'s still good. Let me structure it properly. \n\nWait, let me think again. The user might prefer a different angle. What about their nine lives? Or their love for boxes? Maybe the box one